# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv("../sp_500_stocks.csv")
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.026026099601385244

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = pd.concat(
            [final_dataframe,
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A'
                ],
                index=  my_columns
            ).to_frame().T
            ], 
            ignore_index = True
        )
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,117.83,-0.12761,N/A
1,AAL,14.99,0.061397,N/A
2,AAP,67.44,-0.650552,N/A
3,AAPL,173.965,0.026026,N/A
4,ABBV,148.97,0.101382,N/A
...,...,...,...,...
95,CINF,103.51,0.036017,N/A
96,CL,73.71,-0.081584,N/A
97,CLX,149.98,0.066386,N/A
98,CMA,47.04,-0.414141,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,84,CCL,15.57,0.579534,N/A
1,50,AVGO,842.52,0.55862,N/A
2,65,BKNG,3023.69,0.465523,N/A
3,66,BKR,34.91,0.455299,N/A
4,79,CAT,270.135,0.430617,N/A
5,55,BA,226,0.391174,N/A
6,37,ANET,182.26,0.388769,N/A
7,29,AMAT,144.95,0.380584,N/A
8,24,ALGN,345.34,0.298495,N/A
9,39,ANTM,482.58,0.281524,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')

    try :
        float(portfolio_size)
    except ValueError:
        print ('That is not a number! \nPlease try again:')
        portfolio_size = input('Enter the size of your portfolio:')
portfolio_input()

In [9]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,84,CCL,15.57,0.579534,1
1,50,AVGO,842.52,0.55862,0
2,65,BKNG,3023.69,0.465523,0
3,66,BKR,34.91,0.455299,0
4,79,CAT,270.135,0.430617,0
5,55,BA,226,0.391174,0
6,37,ANET,182.26,0.388769,0
7,29,AMAT,144.95,0.380584,0
8,24,ALGN,345.34,0.298495,0
9,39,ANTM,482.58,0.281524,0


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [10]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]
hqm_dataframe = pd.DataFrame(columns = hqm_columns)
for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    # print (data)
    for symbol in symbol_string.split(','):
        hqm_dataframe = pd.concat(
            [ hqm_dataframe,
            pd.Series(
            [
                symbol,
                data[symbol]['price'],
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A',
                'N/A'
            ],
            index = hqm_columns
            ).to_frame().T
            ], 
            ignore_index = True
        )
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,117.84,N/A,-0.12761,N/A,-0.192821,N/A,-0.072882,N/A,-0.042553,N/A,N/A
1,AAL,14.99,N/A,0.061397,N/A,-0.080685,N/A,0.014845,N/A,-0.137615,N/A,N/A
2,AAP,67.44,N/A,-0.650552,N/A,-0.526937,N/A,-0.403137,N/A,-0.002571,N/A,N/A
3,AAPL,174.05,N/A,0.026026,N/A,0.148499,N/A,-0.001133,N/A,-0.094074,N/A,N/A
4,ABBV,148.97,N/A,0.101382,N/A,0.012497,N/A,0.046134,N/A,0.055911,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
95,CINF,103.54,N/A,0.036017,N/A,-0.164077,N/A,0.004635,N/A,0.012773,N/A,N/A
96,CL,73.71,N/A,-0.081584,N/A,0.01316,N/A,-0.061414,N/A,-0.031818,N/A,N/A
97,CLX,149.98,N/A,0.066386,N/A,0.016082,N/A,-0.068371,N/A,0.002496,N/A,N/A
98,CMA,47.04,N/A,-0.414141,N/A,-0.329485,N/A,0.253301,N/A,-0.106934,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [11]:
time_periods=  [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
]
for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,117.84,N/A,-0.12761,0.25,-0.192821,0.13,-0.072882,0.13,-0.042553,0.49,N/A
1,AAL,14.99,N/A,0.061397,0.62,-0.080685,0.28,0.014845,0.47,-0.137615,0.08,N/A
2,AAP,67.44,N/A,-0.650552,0.01,-0.526937,0.01,-0.403137,0.01,-0.002571,0.64,N/A
3,AAPL,174.05,N/A,0.026026,0.5,0.148499,0.83,-0.001133,0.36,-0.094074,0.19,N/A
4,ABBV,148.97,N/A,0.101382,0.72,0.012497,0.52,0.046134,0.56,0.055911,0.88,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
95,CINF,103.54,N/A,0.036017,0.54,-0.164077,0.15,0.004635,0.41,0.012773,0.78,N/A
96,CL,73.71,N/A,-0.081584,0.31,0.01316,0.53,-0.061414,0.19,-0.031818,0.55,N/A
97,CLX,149.98,N/A,0.066386,0.64,0.016082,0.54,-0.068371,0.15,0.002496,0.72,N/A
98,CMA,47.04,N/A,-0.414141,0.03,-0.329485,0.03,0.253301,0.93,-0.106934,0.14,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [12]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        percentile_col = f'{time_period} Return Percentile'
        momentum_percentiles.append(hqm_dataframe.loc[row, percentile_col])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe
    

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,117.84,N/A,-0.12761,0.25,-0.192821,0.13,-0.072882,0.13,-0.042553,0.49,0.25
1,AAL,14.99,N/A,0.061397,0.62,-0.080685,0.28,0.014845,0.47,-0.137615,0.08,0.3625
2,AAP,67.44,N/A,-0.650552,0.01,-0.526937,0.01,-0.403137,0.01,-0.002571,0.64,0.1675
3,AAPL,174.05,N/A,0.026026,0.5,0.148499,0.83,-0.001133,0.36,-0.094074,0.19,0.47
4,ABBV,148.97,N/A,0.101382,0.72,0.012497,0.52,0.046134,0.56,0.055911,0.88,0.67
...,...,...,...,...,...,...,...,...,...,...,...,...
95,CINF,103.54,N/A,0.036017,0.54,-0.164077,0.15,0.004635,0.41,0.012773,0.78,0.47
96,CL,73.71,N/A,-0.081584,0.31,0.01316,0.53,-0.061414,0.19,-0.031818,0.55,0.395
97,CLX,149.98,N/A,0.066386,0.64,0.016082,0.54,-0.068371,0.15,0.002496,0.72,0.5125
98,CMA,47.04,N/A,-0.414141,0.03,-0.329485,0.03,0.253301,0.93,-0.106934,0.14,0.2825


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [13]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop = True, inplace=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,APA,43.85,N/A,0.26199,0.88,0.183977,0.87,0.341126,0.98,0.164604,1.0,0.9325
1,ANET,182.26,N/A,0.388769,0.94,0.306084,0.95,0.254534,0.94,0.053264,0.86,0.9225
2,BKR,34.91,N/A,0.455299,0.97,0.181734,0.86,0.28325,0.96,0.028864,0.81,0.9
3,AMAT,145.01,N/A,0.380584,0.93,0.24508,0.92,0.129549,0.79,0.064309,0.96,0.9
4,CAT,270.135,N/A,0.430617,0.96,0.116467,0.76,0.281106,0.95,0.048681,0.85,0.88
5,BKNG,3023.69,N/A,0.465523,0.98,0.242249,0.91,0.106019,0.74,0.048305,0.84,0.8675
6,CMCSA,45.69,N/A,0.229083,0.87,0.204547,0.9,0.126894,0.76,0.059014,0.92,0.8625
7,ALGN,345.34,N/A,0.298495,0.92,0.106249,0.74,0.204028,0.89,0.058521,0.9,0.8625
8,ADBE,511.785,N/A,0.195445,0.81,0.423932,1.0,0.3687,0.99,-0.016928,0.62,0.855
9,AKAM,100.86,N/A,0.039777,0.57,0.305304,0.94,0.156182,0.82,0.094598,0.98,0.8275


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [14]:
portfolio_input()

In [15]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])

hqm_dataframe

/tmp/ipykernel_912/3213983214.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,APA,43.85,456,0.26199,0.88,0.183977,0.87,0.341126,0.98,0.164604,1.0,0.9325
1,ANET,182.26,109,0.388769,0.94,0.306084,0.95,0.254534,0.94,0.053264,0.86,0.9225
2,BKR,34.91,572,0.455299,0.97,0.181734,0.86,0.28325,0.96,0.028864,0.81,0.9
3,AMAT,145.01,137,0.380584,0.93,0.24508,0.92,0.129549,0.79,0.064309,0.96,0.9
4,CAT,270.135,74,0.430617,0.96,0.116467,0.76,0.281106,0.95,0.048681,0.85,0.88
5,BKNG,3023.69,6,0.465523,0.98,0.242249,0.91,0.106019,0.74,0.048305,0.84,0.8675
6,CMCSA,45.69,437,0.229083,0.87,0.204547,0.9,0.126894,0.76,0.059014,0.92,0.8625
7,ALGN,345.34,57,0.298495,0.92,0.106249,0.74,0.204028,0.89,0.058521,0.9,0.8625
8,ADBE,511.785,39,0.195445,0.81,0.423932,1.0,0.3687,0.99,-0.016928,0.62,0.855
9,AKAM,100.86,198,0.039777,0.57,0.305304,0.94,0.156182,0.82,0.094598,0.98,0.8275


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [ ]:
writer = pd.ExcelWriter("momentum_strategy.xlsx", engine = "xlsxwriter")
hqm_dataframe.to_excel(writer, sheet_name = "Momentum Strategy", index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:
column_format = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', percent_template]
}

for column in column_format.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_format[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_format[column][0], column_format[column][1])
writer.close()

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [19]:
writer.close()

/workspace/.pyenv_mirror/user/current/lib/python3.11/site-packages/xlsxwriter/workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
